In [ ]:
# Correr para reiniciar Kernel y la sesion.
#!kill -9 -1

# Proyecto Final (Texto - Imagenes)

### Curso Deep Learning y Redes Neuronales - MIIA4406_01

#### Universidad de los Andes

Integrantes:

|Nombre|Codigo|
|---------------------|---------------------|
|Saby Espinel|*201215868*|
|Diego Salazar|*201628925*| <br>
<br>
______________________________________________________________________________________________________________________

# Indice_de_navegacion

1. [Text](#Text)
2. [Images](#Images)

******************************************************************************************************************************************

# Text

******************************************************************************************************************************************

# Librerias (Jupyter y CoLab) >> Text
[Indice de Navegacion](#Indice_de_navegacion)

In [ ]:
# Verificar que se esta usando GPU
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1442079591125361260, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 251789312
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6354184267043998337
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [1]:
##### Para cargar en Jupyter #####
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
#from sklearn.metrics import r2_score, roc_auc_score
#from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from skimage import io

# Text processing
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split

# Keras NN
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, Embedding, LSTM
from keras.optimizers import RMSprop
from keras.callbacks import History
from keras.models import Model
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

%matplotlib inline

C:\Users\da.salazarb\AppData\Local\Continuum\anaconda3\envs\PythonR\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
##### Para usar en CoLab #####
!pip install -U -q PyDrive
!pip install textblob
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files


In [ ]:
##### Para usar en CoLab #####
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Read Data CoLab >> Text
[Indice de Navegacion](#Indice_de_navegacion)

In [ ]:
##### Para cargar en CoLab #####
# Verificacion de contenido de carpetas en el Drive con su respectivo ID
#file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
file_list = drive.ListFile({'q': "'1gX_xAhoYwIwmOVRd33lAuf4pCM3cChB6' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: ExampleTextRF.ipynb, id: 1awW3xWv99qbvcMP1hWTIJjmdmKq8Ku42
title: ExampleImgRF.ipynb, id: 1Pg2-wYIUr866PzKvUy8iXsstZT87uE6x
title: Proyecto 2_birds_DL (1).ipynb, id: 1mRpiAc1wbDI226Zw56g6NvPHK0ebcACn
title: images.zip, id: 1NMIKcqCHJIDaQi4M9mbjrZf0qlpXtrO-
title: images, id: 1pkD2K630_7_sqyt42vnbOYKVZrnyE-VQ
title: Proyecto2.ipynb, id: 131wvpD2jMHusixkRryXoFdTj9DbmbxYp
title: dataTraining.csv, id: 1SKNGe6FFghASG38LjtUaoTrmozUibRCZ
title: dataTesting.csv, id: 1JWWmvheWnofZl2zqY-7Gu2rJprceYB80
title: .ipynb_checkpoints, id: 17KsfZFar28G8dXPBETs6cCpaAknZO3yP
title: moviegenre.png, id: 1h6OU9y8Fwa-HqLMeVDAhLKO9Et6Bojmv
title: README.md, id: 1ICVh_qZvcAXH36lytZyJiYgMFbocGgCd


In [ ]:
##### Para cargar en CoLab #####
trainDrive = drive.CreateFile({'id': '1SKNGe6FFghASG38LjtUaoTrmozUibRCZ'})
trainDrive.GetContentFile('dataTraining.csv')
dataTraining=pd.read_csv("dataTraining.csv", index_col=0)

testDrive = drive.CreateFile({'id': '1JWWmvheWnofZl2zqY-7Gu2rJprceYB80'})
testDrive.GetContentFile('dataTesting.csv')
dataTesting=pd.read_csv("dataTesting.csv", index_col=0)

# Read Data Jupyter >> Text
[Indice de Navegacion](#Indice_de_navegacion)

In [2]:
##### Para cargar en Jupyter #####
path = 'C:/Users/da.salazarb/Google Drive (dasalazarb@unal.edu.co)/AppliedDeepLearningClass-master/finalProject'
dataTraining = pd.read_csv(os.path.join(path, 'dataTraining.csv'), encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv(os.path.join(path, 'dataTesting.csv'), encoding='UTF-8', index_col=0)
#print(dataTraining.head())
#print(dataTesting.head())

******************************************************************************************************************************************

# Cleaning >> Text
[Indice de Navegacion](#Indice_de_navegacion)

In [3]:
# Definicion de funcion para preprocesar el texto ->
# (dataTraining['post'] and dataTesting['post'])
def preProssText(df, commonWord, numberCommonWords, rareWord, numberRareWords, spellCorrect):
    # Remove punctutation
    df['plot'] = df['plot'].str.replace('[^\w\s]','')
    
    # Lower case
    df['plot'] = df['plot'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    
    # Remove Common words
    if commonWord == True:
        freq = pd.Series(' '.join(df['plot']).split()).value_counts()[:numberCommonWords]
        df['plot'] = df['plot'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
    
    # Remove Rare words
    if rareWord == True:
        freq = pd.Series(' '.join(df['plot']).split()).value_counts()[-numberRareWords:]
        df['plot'] = df['plot'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
    
    # Spelling correction -> Tener cuidado puede cambiar los valores. Ej: "ur" por "or" en vez de "your"
    if spellCorrect == True:
        from textblob import TextBlob
        df['plot'] = df['plot'].apply(lambda x: str(TextBlob(x).correct()))
    
    return df

In [4]:
# Pre-procesamiento de texto
dataTraining = preProssText(dataTraining, commonWord=True, numberCommonWords=10, rareWord=True, numberRareWords=10, spellCorrect=False)
dataTesting = preProssText(dataTesting, commonWord=True, numberCommonWords=10, rareWord=True, numberRareWords=10, spellCorrect=False)

In [5]:
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most story single father who takes eight year ...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,serial killer decides teach secrets satisfying...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,sweden female blackmailer with disfiguring fac...,"['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,friday afternoon new york president tredway co...,['Drama'],7.4
2582,1990,Narrow Margin,los angeles editor publishing house carol hunn...,"['Action', 'Crime', 'Thriller']",6.6


In [6]:
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,who meets by fate shall be sealed by fate ther...
4,1978,Midnight Express,true story billy hayes an american college stu...
5,1996,Primal Fear,martin vail left chicago da office become succ...
6,1950,Crisis,husband wife americans dr eugene mrs helen fer...
7,1959,The Tingler,coroner scientist dr warren chapin researching...


***************************************************************************************************

# Create count vectorizer with ngrams  >> Text
[Indice de Navegacion](#Indice_de_navegacion)

In [7]:
# define a function that accepts text and returns a list of lemmas
import nltk
nltk.download('wordnet')

def split_into_lemmas(text):
    from nltk.stem import WordNetLemmatizer
    wordnet_lemmatizer = WordNetLemmatizer()
    
    text = text.lower()
    words = text.split()
    return [wordnet_lemmatizer.lemmatize(word) for word in words]

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\da.salazarb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
# define a function that accepts text and returns a list of stem
def split_into_stem(text):
    stemmer = SnowballStemmer('english')
    text = text.lower()
    words = text.split()
    return [stemmer.stem(word) for word in words]

In [9]:
vect = CountVectorizer(analyzer=split_into_lemmas, ngram_range=(1, 2), max_features=1000, stop_words='english')
X_Text = vect.fit_transform(dataTraining['plot'])
X_Text.shape

(7895, 1000)

In [10]:
print(vect.get_feature_names()[:50])

['a', 'ability', 'able', 'about', 'accepts', 'accident', 'accidentally', 'across', 'act', 'action', 'actor', 'actress', 'actually', 'adam', 'adult', 'adventure', 'affair', 'after', 'again', 'against', 'age', 'agent', 'ago', 'agrees', 'air', 'alan', 'alex', 'alice', 'alien', 'alive', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'america', 'american', 'among', 'an', 'angeles', 'animal', 'ann', 'anna', 'another', 'answer', 'any', 'anyone']


***************************************************************************************************

# Create y_genres >> Text
[Indice de Navegacion](#Indice_de_navegacion)

In [11]:
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most story single father who takes eight year ...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,serial killer decides teach secrets satisfying...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,sweden female blackmailer with disfiguring fac...,"['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,friday afternoon new york president tredway co...,['Drama'],7.4
2582,1990,Narrow Margin,los angeles editor publishing house carol hunn...,"['Action', 'Crime', 'Thriller']",6.6


In [12]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [13]:
y_genres.shape

(7895, 24)

******************************************************************************************************************************************

# Split train and test >> Text
[Indice de Navegacion](#Indice_de_navegacion)

In [14]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_Text, y_genres, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train_genres.shape, y_test_genres.shape

((6316, 1000), (1579, 1000), (6316, 24), (1579, 24))

******************************************************************************************************************************************

# Train DNN >> Text
[Indice de Navegacion](#Indice_de_navegacion)

In [15]:
# Modelo
top_words = 1000
max_review_length = 1000
# Set the model
def textModel(max_review_length):
    model = Sequential()
    model.add(Embedding(top_words, 32, input_length=max_review_length))
    model.add(BatchNormalization())
    model.add(LSTM(256, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(LSTM(100))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(24, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0), metrics=['accuracy'])
    return model

In [16]:
ix_train=np.random.choice(X_train.shape[0], 3000)
ix_test=np.random.choice(X_test.shape[0], 50)
model=textModel(max_review_length)
model.fit(X_train[ix_train], y_train_genres[ix_train], epochs=3, batch_size=50, validation_data=(X_test[ix_test], y_test_genres[ix_test]))
#model.fit(X_Text, y_genres, epochs=3, batch_size=1500)

Train on 3000 samples, validate on 50 samples
Epoch 1/3
3000/3000 [==============================] - 413s 138ms/step - loss: 9.0918 - acc: 0.0583 - val_loss: 9.2840 - val_acc: 0.0600
Epoch 2/3
3000/3000 [==============================] - 419s 140ms/step - loss: 8.5240 - acc: 0.0883 - val_loss: 8.1474 - val_acc: 0.3000
Epoch 3/3
3000/3000 [==============================] - 421s 140ms/step - loss: 7.9842 - acc: 0.1777 - val_loss: 8.0731 - val_acc: 0.3600


# Prediction on dataTesting dataset

In [ ]:
X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = model.predict_proba(X_test_dtm)

pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols).to_csv('pred_genres_text_RF.csv', index_label='ID')

******************************************************************************************************************************************

# Images

******************************************************************************************************************************************

# Librerias (Jupyter y CoLab) >> Image
[Indice de Navegacion](#Indice_de_navegacion)

In [17]:
##### Para cargar en Jupyter #####
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
#from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from skimage import io
from tqdm import tqdm

## Image transformation and reading
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import cv2
from keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dropout, Dense, Input, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Model, Sequential

%matplotlib inline

In [ ]:
##### Para cargar en Colab #####
!pip install -U -q PyDrive
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

In [ ]:
##### Para cargar en Colab #####
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
##### Para cargar en Colab #####
# Tome las primeras 1025 + 1055 imagenes
!unzip -o images.zip

In [ ]:
##### Para cargar en Colab #####
path='images/'
#image = io.imread(os.path.join(path, 'images_resize_gray', str(dataTraining.index[0]) + '_resize_gray.jpeg'))
image = io.imread(os.path.join(path, '1713.jpeg'))

******************************************************************************************************************************************

# Read Data CoLab >> Image
[Indice de Navegacion](#Indice_de_navegacion)

In [ ]:
# Verificacion de contenido de carpetas en el Drive con su respectivo ID
#file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
file_list = drive.ListFile({'q': "'1gX_xAhoYwIwmOVRd33lAuf4pCM3cChB6' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

In [ ]:
# Download a file based on its file ID.
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1NMIKcqCHJIDaQi4M9mbjrZf0qlpXtrO-' # -> cnn_data.zip
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile(downloaded['title'])

In [ ]:
def padImages(desired_size, carpeta, path):
    #desired_size=200
    #imgFinal = np.ones((1,200,200,3))
    imgFinal = np.ones((1,desired_size,desired_size,3))
    for i in os.listdir(carpeta):
    #for i in os.listdir('images'):
        if i.startswith('desktop'):
            pass
        else:
            #images_training.append(load_img(os.path.join(path, str(i)),grayscale=False,target_size=None,interpolation='nearest'))
            #image=load_img(os.path.join(path, str(i)),grayscale=False,target_size=None,interpolation='nearest')
            #image=img_to_array(image)

            im = cv2.imread(os.path.join(path, str(i)))
            old_size = im.shape[:2] # old_size is in (height, width) format

            ratio = float(desired_size)/max(old_size)
            new_size = tuple([int(x*ratio) for x in old_size])

            # new_size should be in (width, height) format

            im = cv2.resize(im, (new_size[1], new_size[0]))

            delta_w = desired_size - new_size[1]
            delta_h = desired_size - new_size[0]
            top, bottom = delta_h//2, delta_h-(delta_h//2)
            left, right = delta_w//2, delta_w-(delta_w//2)

            color = [0, 0, 0]
            new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
            new_im=new_im.reshape((1,new_im.shape[0],new_im.shape[1],new_im.shape[2]))
            imgFinal = np.concatenate([imgFinal, new_im], axis=0)
    return imgFinal

# Code for padding: https://jdhao.github.io/2017/11/06/resize-image-to-square-with-padding/

In [ ]:
#imgFinal_2=np.rollaxis(imgFinal, 3, 1) #Para cuando le de la locura al ImageDataGenerator
imgFinal = padImages(desired_size=200, carpeta="images", path=path)
imgFinal=np.delete(imgFinal, 0, 0)
imgFinal.shape
datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
imgFinal_2=imgFinal.reshape((imgFinal.shape[0],200,200,3))
datagen.fit(imgFinal_2)

# Create y_genres Colab >> Image
[Indice de Navegacion](#Indice_de_navegacion)

In [ ]:
# Verificacion de contenido de carpetas en el Drive con su respectivo ID
#file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
file_list = drive.ListFile({'q': "'1SKNGe6FFghASG38LjtUaoTrmozUibRCZ' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

In [ ]:
trainDrive = drive.CreateFile({'id': '1SKNGe6FFghASG38LjtUaoTrmozUibRCZ'})
trainDrive.GetContentFile('dataTraining.csv')
dataTraining=pd.read_csv("dataTraining.csv", index_col=0)

testDrive = drive.CreateFile({'id': '1JWWmvheWnofZl2zqY-7Gu2rJprceYB80'})
testDrive.GetContentFile('dataTesting.csv')
dataTesting=pd.read_csv("dataTesting.csv", index_col=0)

In [ ]:
## ix_movies guarda los indices que contiene la carpeta images/
ix_movies = []
for i in range(0,len(os.listdir("images/"))):
    if os.listdir("images/")[i] == "desktop.ini":
        pass
    else:
        ix_movies.append(int(os.listdir("images/")[i].replace(".jpeg","")))

In [ ]:
# Se discrimina de las imagenes contenidas en dataTraining
# 
imgTrain=[]
data_training=[]
for i in range(0,len(ix_movies)):
    try:
        data_training.append(dataTraining.loc[ix_movies[i]])
        imgTrain.append(i)
    except:
        pass
data_training=pd.DataFrame(data_training)

In [ ]:
data_training['genres'] = data_training['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(data_training['genres'])

******************************************************************************************************************************************

# Split train and test CoLab  >> Image
[Indice de Navegacion](#Indice_de_navegacion)

In [ ]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(imgTraining, y_genres, test_size=0.33, random_state=42)
X_train.shape, X_test.shape, y_train_genres.shape, y_test_genres.shape

******************************************************************************************************************************************

# Read Data Jupyter  >> Image
[Indice de Navegacion](#Indice_de_navegacion)

In [18]:
# Training
images_training = []
for i in tqdm(dataTraining.index):
    path ="C:/Users/da.salazarb/Google Drive (dasalazarb@unal.edu.co)/AppliedDeepLearningClass-master/finalProject/"
    #images_training.append(io.imread(os.path.join(path, 'images_resize_gray', str(i) + '_resize_gray.jpeg')).flatten())
    images_training.append(io.imread(os.path.join(path, 'images_resize_gray', str(i) + '_resize_gray.jpeg')))
    #images_training.append(io.imread(os.path.join(path, str(i) + ".jpeg")).flatten())
images_training = np.stack(images_training)
images_training.shape, dataTraining.shape[0]

100%|██████████| 7895/7895 [00:10<00:00, 744.60it/s]


((7895, 256, 160), 7895)

In [19]:
# Testing
images_testing  = []
for i in tqdm(dataTesting.index):
    #images_testing.append(io.imread(os.path.join(path, 'images_resize_gray', str(i) + '_resize_gray.jpeg')).flatten())
    images_testing.append(io.imread(os.path.join(path, 'images_resize_gray', str(i) + '_resize_gray.jpeg')))
images_testing = np.stack(images_testing)
images_testing.shape, dataTesting.shape[0]

100%|██████████| 3383/3383 [00:04<00:00, 803.64it/s]


((3383, 256, 160), 3383)

# Apply PCA  >> Image
[Indice de Navegacion](#Indice_de_navegacion)

In [ ]:
#pca = PCA(n_components=32)
#images_training_pca = pca.fit_transform(images_training)

In [ ]:
#images_testing_pca = pca.transform(images_testing)

# Create genres Jupyter >> Image
[Indice de Navegacion](#Indice_de_navegacion)

In [7]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

******************************************************************************************************************************************

# Split train and test CoLab  >> Image
[Indice de Navegacion](#Indice_de_navegacion)

In [20]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(images_training, y_genres, test_size=0.33, random_state=42)
#X_train, X_test, y_train_genres, y_test_genres = train_test_split(images_training_pca, y_genres, test_size=0.33, random_state=42)
X_train.shape, X_test.shape, y_train_genres.shape, y_test_genres.shape 

((5289, 256, 160), (2606, 256, 160), (5289, 24), (2606, 24))

In [21]:
X_train=X_train.reshape((X_train.shape[0], 1, X_train.shape[1], X_train.shape[2]))
X_test=X_test.reshape((X_test.shape[0], 1, X_test.shape[1], X_test.shape[2]))
X_train.shape, X_test.shape

((5289, 1, 256, 160), (2606, 1, 256, 160))

In [42]:
# Esto es para elegir un tamaño menor de muestras
ix_train=np.random.choice(X_train.shape[0], 2000)
ix_test=np.random.choice(X_test.shape[0], 50)

X_train=X_train[ix_train]
X_test=X_test[ix_test]
y_train_genres=y_train_genres[ix_train]
y_test_genres=y_test_genres[ix_test]
X_train.shape, X_test.shape, y_train_genres.shape, y_test_genres.shape 

((2000, 1, 256, 160), (50, 1, 256, 160), (2000, 24), (50, 24))

******************************************************************************************************************************************

# Train DNN  >> Image
[Indice de Navegacion](#Indice_de_navegacion)

In [29]:
from keras.applications.vgg19 import VGG19
base_modelVGG19 = VGG19(weights='imagenet') #, input_shape=(1,256, 160)
base_modelVGG19.compile(optimizer="adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(images_training, y_genres, test_size=0.33, random_state=42)
X_train=X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
X_test=X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))
X_train.shape, X_test.shape

# Model 1

In [24]:
def preTrainedModel(base_model):
    input_shape
    x = base_model.output
    x = Dense(256)(x)
    x = Activation("tanh")(x)
    x = Dense(23)(x)
    x = Activation("sigmoid")(x)
    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer="rmsprop", loss = "categorical_crossentropy", metrics = ["accuracy"])
    return model

In [25]:
model = preTrainedModel(base_modelVGG19)
model.fit(X_train, y_train_genres, epochs=5, validation_data=[X_test, y_test_genres])

ValueError: Error when checking input: expected input_1 to have shape (224, 224, 3) but got array with shape (1, 256, 160)

# Fine Tunning >> Image
[Indice de Navegacion](#Indice_de_navegacion)

In [ ]:
def fineTune(base_model):
    x = base_model.output
    x = Dense(256)(x)
    x = Activation("tanh")(x)
    x = Dense(23)(x)
    x = Activation("sigmoid")(x)
    model = Model(inputs=base_model.input, outputs=x)
    #model.compile(optimizer="adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
    return model

modeltune = fineTune(base_modelVGG19)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_modelVGG19.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
modeltune.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics = ["accuracy"])

# train the model on the new data for a few epochs
modeltune.fit(X_train, y_train_genres, epochs=5, validation_data=[X_test, y_test_genres])

# Vamos a escoger las layers de MobileNet para entrenarlas.
for layer in modeltune.layers[:25]:
    layer.trainable = False
for layer in modeltune.layers[25:]:
    layer.trainable = True

# Re-compilamos el modelos para que estas modificaciones tengan efecto
# Usamos SGD y un learning rate bajo
from keras.optimizers import SGD
modeltune.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics = ["accuracy"])

# Entrenamos el modelo nuevamente Esta vez con todo 
#(esta vez ajustando los 2 primeros bloques de inicio junto a las capas superiores de Dense)
modeltune.fit(X_train, y_train_genres, epochs=5, validation_data=[X_test, y_test_genres])

# Model 2

In [43]:
def model2(activation="tanh", optimizer="rmsprop", drop=.5, neuron=256):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding="valid", data_format="channels_first", input_shape=(1,256, 160)))
    model.add(Activation(activation))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(1, 1)))
    model.add(Dropout(drop))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation(activation))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(1, 1)))
    model.add(Dropout(drop))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation(activation))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(1, 1)))
    model.add(Dropout(drop))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(neuron))
    model.add(Activation(activation))
    model.add(Dropout(0.5))
    model.add(Dense(24))
    model.add(Activation('sigmoid'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

In [44]:
model = model2()
model.fit(X_train, y_train_genres, epochs=2, batch_size=50, validation_data=[X_test, y_test_genres])

Train on 2000 samples, validate on 50 samples
Epoch 1/2
2000/2000 [==============================] - 824s 412ms/step - loss: 8.4900 - acc: 0.2885 - val_loss: 7.8602 - val_acc: 0.3600
Epoch 2/2
2000/2000 [==============================] - 810s 405ms/step - loss: 7.7821 - acc: 0.2615 - val_loss: 8.0920 - val_acc: 0.1000


# Prediction on dataTesting dataset

In [ ]:
images_testing = images_testing.reshape(images_testing.shape[0], 1, images_testing.shape[1], images_testing.shape[2])

In [45]:
cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = model.predict_proba(images_testing)

pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols).to_csv('pred_genres_img_RF.csv', index_label='ID')

In [93]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [95]:
# Definicion del modelo KerasClassifier
model = KerasClassifier(build_fn=model2, verbose=0)

# Lista de parametros a evaluar
parametros = {
    "activation": ["softmax", "elu", "selu", "softplus", "softsign", "relu" ,"tanh", "sigmoid", "hard_sigmoid", "linear"],
    "optimizer": ["SGD", "RMSprop", "Adagrad", "Adadelta", "Adam", "Adamax", "Nadam"], 
    "drop": [0.2,0.4,0.6,0.8,1],
    #"kernel": ["random_normal", "he_uniform", "lecun_normal", "he_normal", "glorot_uniform", "glorot_normal", "lecun_uniform", "zeros", "ones"], 
    "neuron": [5,10,64,256]
    #"moreHiddenLayers": [True, False]
}

# Busqueda aleatoria entre los parametros
random_search = RandomizedSearchCV(estimator=model, param_distributions=parametros, n_jobs=-1)

In [ ]:
# Este proceso lleva tiempo por lo que se corre en una linea aparte
# Realiza modelos de manera aleatoria, esto nos permite ver un espectro de cuales paramatros
# y en que rangos se deben fijar los valores.
random_result=random_search.fit(X_train, y_train_genres, epochs=3, batch_size=50, validation_data=[X_test, y_test_genres])

In [ ]:
random_result._bestParams

# Apply models to kaggle test
[Indice de Navegacion](#Indice_de_navegacion)

In [ ]:
X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = model.predict_proba(X_test_dtm)

pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols).to_csv('pred_genres_text_RF.csv', index_label='ID')